In [20]:
import pandas as pd
import datetime as dt
#  import numpy as np
import json
import requests
from sqlalchemy import create_engine

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import os
# Set up directories
input_dir = 'input_data'
landing_dir = 'landing_data'
cleaned_dir = 'cleaned_data'

PSQL_USER='toddh'
PSQL_PWD='heritage'

In [2]:
INPUT_FILE = os.path.join( landing_dir, 'pol_13_16_df.csv')

pol_13_16_df = pd.read_csv(INPUT_FILE)
pol_13_16_df.head()

,State,State_Abbr,id,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,Alabama,AL,0.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.259091,0.80,2013.0
1,Alabama,AL,1.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.266667,0.50,2013.0
2,Alabama,AL,4.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.305625,0.47,2013.0
3,Alabama,AL,5.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.400000,0.50,2013.0
4,Alabama,AL,8.0,2013-12-03,Parts per billion,14.912500,Parts per million,0.008167,Parts per billion,0.305263,11.0,3.0,Parts per million,0.216818,0.35,2013.0


In [3]:
# drop columns not needed - I hope.
pol_13_16_df.drop(columns=['State', 'id', 'Date_Local', 'NO2_Units', 'NO2_Mean', 'SO2_Units', 'SO2_Mean', \
                           'SO2 1st Max Hour', 'SO2 AQI', 'CO_Units', 'CO_Mean', 'CO 1st Max Value'], inplace=True)
pol_13_16_df.head()

,State_Abbr,O3_Units,O3_Mean,year
0,AL,Parts per million,0.013542,2013.0
1,AL,Parts per million,0.013542,2013.0
2,AL,Parts per million,0.009375,2013.0
3,AL,Parts per million,0.009375,2013.0
4,AL,Parts per million,0.008167,2013.0


Go thru and generate stats off of pol_13_16_df dataframe

In [4]:
# df.groupby(['col5','col2']).reset_index()

# pol_groupby_min_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).min()\
#    .groupby('State_Abbr', as_index=False)
pol_groupby_min_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).min()

pol_groupby_min_df.head()

,year,State_Abbr,O3_Units,O3_Mean
0,2013.0,AL,Parts per million,0.000000
1,2013.0,AR,Parts per million,0.003292
2,2013.0,AZ,Parts per million,0.002000
3,2013.0,CA,Parts per million,0.000000
4,2013.0,CO,Parts per million,0.000083


In [5]:
# - get max

pol_groupby_max_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).max()
pol_groupby_max_df.head()


,year,State_Abbr,O3_Units,O3_Mean
0,2013.0,AL,Parts per million,0.026333
1,2013.0,AR,Parts per million,0.049000
2,2013.0,AZ,Parts per million,0.059750
3,2013.0,CA,Parts per million,0.071167
4,2013.0,CO,Parts per million,0.057100


In [6]:
# - mean

pol_groupby_mean_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).mean()
pol_groupby_mean_df.head(11)

,year,State_Abbr,O3_Mean
0,2013.0,AL,0.012099
1,2013.0,AR,0.025577
2,2013.0,AZ,0.027973
3,2013.0,CA,0.026075
4,2013.0,CO,0.028709
5,2013.0,CT,0.031195
6,2013.0,DC,0.025497
7,2013.0,DE,0.028941
8,2013.0,FL,0.026027
9,2013.0,GA,0.019709


Merge the dataframes

In [7]:
# - 


In [8]:
stats_df = pd.merge(pol_groupby_min_df, pol_groupby_max_df, how='left', left_on=['year', 'State_Abbr'], right_on=['year', 'State_Abbr'], suffixes=("_min", "_max"))
stats_df.head()

,year,State_Abbr,O3_Units_min,O3_Mean_min,O3_Units_max,O3_Mean_max
0,2013.0,AL,Parts per million,0.000000,Parts per million,0.026333
1,2013.0,AR,Parts per million,0.003292,Parts per million,0.049000
2,2013.0,AZ,Parts per million,0.002000,Parts per million,0.059750
3,2013.0,CA,Parts per million,0.000000,Parts per million,0.071167
4,2013.0,CO,Parts per million,0.000083,Parts per million,0.057100


In [9]:
# Clean up the columns in stats_df
stats_df.drop(columns=['O3_Units_max'])


,year,State_Abbr,O3_Units_min,O3_Mean_min,O3_Mean_max
0,2013.0,AL,Parts per million,0.000000,0.026333
1,2013.0,AR,Parts per million,0.003292,0.049000
2,2013.0,AZ,Parts per million,0.002000,0.059750
3,2013.0,CA,Parts per million,0.000000,0.071167
4,2013.0,CO,Parts per million,0.000083,0.057100
5,2013.0,CT,Parts per million,0.000958,0.067250
6,2013.0,DC,Parts per million,0.000125,0.055917
7,2013.0,DE,Parts per million,0.001667,0.047833
8,2013.0,FL,Parts per million,0.008583,0.049042
9,2013.0,GA,Parts per million,0.002875,0.047250


In [10]:
stats_df = pd.merge(stats_df, pol_groupby_mean_df)
stats_df.head()

,year,State_Abbr,O3_Units_min,O3_Mean_min,O3_Units_max,O3_Mean_max,O3_Mean
0,2013.0,AL,Parts per million,0.000000,Parts per million,0.026333,0.012099
1,2013.0,AR,Parts per million,0.003292,Parts per million,0.049000,0.025577
2,2013.0,AZ,Parts per million,0.002000,Parts per million,0.059750,0.027973
3,2013.0,CA,Parts per million,0.000000,Parts per million,0.071167,0.026075
4,2013.0,CO,Parts per million,0.000083,Parts per million,0.057100,0.028709


In [11]:
# finalize column names
stats_df.rename(columns={"O3_Units_min": "O3_Units", "O3_Mean_min": "O3_min", "O3_Mean_max": "O3_max"})

,year,State_Abbr,O3_Units,O3_min,O3_Units_max,O3_max,O3_Mean
0,2013.0,AL,Parts per million,0.000000,Parts per million,0.026333,0.012099
1,2013.0,AR,Parts per million,0.003292,Parts per million,0.049000,0.025577
2,2013.0,AZ,Parts per million,0.002000,Parts per million,0.059750,0.027973
3,2013.0,CA,Parts per million,0.000000,Parts per million,0.071167,0.026075
4,2013.0,CO,Parts per million,0.000083,Parts per million,0.057100,0.028709
5,2013.0,CT,Parts per million,0.000958,Parts per million,0.067250,0.031195
6,2013.0,DC,Parts per million,0.000125,Parts per million,0.055917,0.025497
7,2013.0,DE,Parts per million,0.001667,Parts per million,0.047833,0.028941
8,2013.0,FL,Parts per million,0.008583,Parts per million,0.049042,0.026027
9,2013.0,GA,Parts per million,0.002875,Parts per million,0.047250,0.019709


In [12]:
stats_df.drop(columns=['O3_Units_max'])
stats_df.head()

,year,State_Abbr,O3_Units_min,O3_Mean_min,O3_Units_max,O3_Mean_max,O3_Mean
0,2013.0,AL,Parts per million,0.000000,Parts per million,0.026333,0.012099
1,2013.0,AR,Parts per million,0.003292,Parts per million,0.049000,0.025577
2,2013.0,AZ,Parts per million,0.002000,Parts per million,0.059750,0.027973
3,2013.0,CA,Parts per million,0.000000,Parts per million,0.071167,0.026075
4,2013.0,CO,Parts per million,0.000083,Parts per million,0.057100,0.028709


In [13]:
# before writing out file convert Year to int so now dot zero (.0) is present in file
stats_df['year'] = pd.to_numeric(stats_df['year'], downcast='integer')
stats_df.head()

,year,State_Abbr,O3_Units_min,O3_Mean_min,O3_Units_max,O3_Mean_max,O3_Mean
0,2013,AL,Parts per million,0.000000,Parts per million,0.026333,0.012099
1,2013,AR,Parts per million,0.003292,Parts per million,0.049000,0.025577
2,2013,AZ,Parts per million,0.002000,Parts per million,0.059750,0.027973
3,2013,CA,Parts per million,0.000000,Parts per million,0.071167,0.026075
4,2013,CO,Parts per million,0.000083,Parts per million,0.057100,0.028709


In [14]:
stats_df=stats_df.rename(columns={'year':'Year', 'State_Abbr':'State', 'O3_Units_min':'O3_Units_Measurement', 'O3_Mean_min':'O3_min', 'O3_Mean_max':'O3_max'})
stats_df=stats_df.drop(['O3_Units_max'], axis=1)
stats_df.head()

,Year,State,O3_Units_Measurement,O3_min,O3_max,O3_Mean
0,2013,AL,Parts per million,0.000000,0.026333,0.012099
1,2013,AR,Parts per million,0.003292,0.049000,0.025577
2,2013,AZ,Parts per million,0.002000,0.059750,0.027973
3,2013,CA,Parts per million,0.000000,0.071167,0.026075
4,2013,CO,Parts per million,0.000083,0.057100,0.028709


In [15]:
# save to a text file in case of problems with loading
OUTPUT_FILE = os.path.join( cleaned_dir, 'cleaned_pollution.csv')
stats_df.to_csv(OUTPUT_FILE)

In [21]:
# load to the database

rds_connection_string = f'{PSQL_USER}:{PSQL_PWD}@localhost:5432/pollution'
engine = create_engine(f'postgresql://{rds_connection_string}')

print(PSQL_USER)
print(PSQL_PWD)

toddh
heritage


In [22]:
stats_df.to_sql(name='cleaned_pollution', con=engine, if_exists='append', index=False)